<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E3%80%90Demo06%E3%80%91%E7%94%A8_RAG_%E6%89%93%E9%80%A0%E5%BF%83%E9%9D%88%E8%99%95%E6%96%B9%E7%B1%A4%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入需要的套件

這裡主要用 `LangChain`, 這可以說整合各式 LLM 功能的方便套件。

In [2]:
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [3]:
!pip install langchain langchain-community openai faiss-cpu unstructured tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━

讀入正確的 `nltk` 所需資料。

In [4]:
import nltk

In [5]:
nltk.data.path.append("/root/nltk_data")
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

讀入一大票需要的函式。

In [6]:
import os
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

### 2. 讀入範例資料

我們這裡用聖嚴法師的《真正的快樂》一書為範例, 當然其實可以用更多的資料, 直接放入相對的資料夾 (這裡是設在 `books` 之下) 即可。包括這本書都在[《法鼓全集》](https://ddc.shengyen.org/)之下, 請注意版權屬「法鼓文化」所有, 我們只是作為範例。

In [7]:
# 下載 books.zip 檔案
!wget -O books.zip https://github.com/yenlung/AI-Demo/raw/refs/heads/master/books.zip

# 解壓縮 books.zip 到 books 資料夾
!unzip -o books.zip

--2024-10-31 08:40:25--  https://github.com/yenlung/AI-Demo/raw/refs/heads/master/books.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yenlung/AI-Demo/refs/heads/master/books.zip [following]
--2024-10-31 08:40:25--  https://raw.githubusercontent.com/yenlung/AI-Demo/refs/heads/master/books.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63072 (62K) [application/zip]
Saving to: ‘books.zip’

books.zip           100%[===================>]  61.59K  --.-KB/s    in 0.007s  

2024-10-31 08:40:26 (9.02 MB/s) - ‘books.zip’ saved [63072/63072]

Archive:  books.zip
   creating: books/
  inflating: 

### 3. 設定 OpenAI 金鑰

In [8]:
from getpass import getpass

In [9]:
api_key = getpass("請輸入您的 OpenAI API key: ")

請輸入您的 OpenAI API key: ··········


In [ ]:
# from google.colab import userdata
# # 將 OpenAI API Key 記在 colab 當中
# api_key = userdata.get('keyFor108')

In [10]:
os.environ["OPENAI_API_KEY"] = api_key

### 4. 建立向量資料庫

#### Step 1: 加載資料夾中的文件

In [11]:
loader = DirectoryLoader("books", glob="*.txt")  # 替換為你的資料夾路徑
documents = loader.load()

#### Step 2: 將文件分割成較小的片段

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

#### Step 3: 使用 OpenAI 的嵌入來將文件轉為向量嵌入

In [13]:
embeddings = OpenAIEmbeddings()

<ipython-input-13-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


#### Step 4: 使用 FAISS 建立向量資料庫

In [14]:
vector_store = FAISS.from_documents(split_docs, embeddings)

#### Step 5: 建立檢索器

In [15]:
retriever = vector_store.as_retriever()

### 5. 打造心靈處方籤機器人

#### 選定語言模型

In [16]:
llm = ChatOpenAI(model="gpt-4o")

<ipython-input-16-e8ae6cf3228a>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")


#### 定義一些心靈處方籤

In [17]:
spiritual_prescriptions = [
    "感謝給我們機會，順境、逆境，皆是恩人。",
    "身心常放鬆，逢人面帶笑；放鬆能使我們身心健康，帶笑容易增進彼此友誼。",
    "識人識己識進退，時時身心平安；知福惜福多培福，處處廣結善緣。",
    "平常心就是最自在、最愉快的心。",
    "知道自己的缺點愈多，成長的速度愈快，對自己的信心也就愈堅定。"
]

#### 建立一個結合檢索與生成的 RAG 問答鏈

In [18]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

#### 定義真正的心靈處方籤主函式

注意最主要還是設計 `prompt` 的型式。

In [19]:
def answer_user_question(question):
    # 抽取一條隨機的心靈處方籤
    chosen_prescription = np.random.choice(spiritual_prescriptions)

    # 檢索資料夾中的相關內容
    retriever_result = qa_chain.run(question)

    print(f"你抽到的心靈處方籤: {chosen_prescription}")

    # 自訂 prompt，結合心靈處方籤、上下文和使用者問題
    prompt = f"""
    使用者抽到了一個心靈處方籤，它的內容是：{chosen_prescription}

    以下是我們從資料庫中檢索到的內容，這些內容來自書中的資料，並與使用者的問題相關：
    {retriever_result}

    請根據「心靈處方籤」中的訊息，結合書中的資料，用類似的語氣和觀念來回應使用者的問題：
    「{question}」
    """

    # 使用 HumanMessage 包裝 prompt 並生成回答
    final_response = llm.invoke([HumanMessage(content=prompt)])

    return final_response.content

In [20]:
# 使用範例：回答一個使用者問題
user_question = "今天有颱風好可怕，該如何面對？"
response = answer_user_question(user_question)

print(f'\n經過機器人得到的內容是 \n==================== \n{response}')

<ipython-input-19-c147d0f04a9d>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever_result = qa_chain.run(question)


你抽到的心靈處方籤: 平常心就是最自在、最愉快的心。

經過機器人得到的內容是 
面對颱風這樣的自然災害，我們可以從「平常心就是最自在、最愉快的心」中獲得啟發。保持平常心，讓自己能冷靜思考和有效行動，這樣可以更自在地應對挑戰。以下是一些建議來幫助您面對颱風：

1. **提前準備**：在颱風來臨前做好準備，將物品妥善收納，這樣可以讓您在面對風雨時更加安心。

2. **保持冷靜**：以平常心應對颱風，讓自己保持冷靜和沉著，這樣您能夠更清晰地思考和做決策。

3. **依賴信仰的力量**：如果您有信仰，可以通過禱告或誦念經文來獲得心靈的平和，這樣能讓您在風雨中保持穩定。

4. **接受現實**：有時候損害是無法避免的，接受現實並專注於災後的處理工作，這樣能讓您更有效地面對挑戰。

5. **遵循安全指引**：確保您和家人的安全是最重要的，聽從政府的指引，這樣能讓您更有信心地度過颱風。

記住，颱風過後總會有晴天，保持希望和信心，讓自己以平常心迎接未來的每一天。


# 建立 Gradio 互動介面

In [21]:
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [29]:
def answer_user_question(question):
    # 抽取一條隨機的心靈處方籤
    chosen_prescription = np.random.choice(spiritual_prescriptions)

    # 檢索資料夾中的相關內容
    retriever_result = qa_chain.run(question)

    # 自訂 prompt，結合心靈處方籤、上下文和使用者問題
    prompt = f"""
    使用者抽到了一個心靈處方籤，它的內容是：{chosen_prescription}

    以下是我們從資料庫中檢索到的內容，這些內容來自書中的資料，並與使用者的問題相關：
    {retriever_result}

    請根據「心靈處方籤」中的訊息，結合書中的資料，用類似的語氣和觀念來回應使用者的問題：
    「{question}」
    """

    # 使用 HumanMessage 包裝 prompt 並生成回答
    final_response = llm.invoke([HumanMessage(content=prompt)])

    return chosen_prescription, final_response.content

In [30]:
title = "【拍拍機器人】AI 心靈處方籤"
description = "請你注意自己的呼吸一分鐘, 寫下你心裡浮現的問題。發送之後會幫你抽出一支心靈處方籤, 還會幫你解籤 :)"

In [31]:
inp = gr.Textbox(label="請寫下你的問題:")
out1 = gr.Textbox(label="你抽到的心靈處方籤")
out2 = gr.Textbox(label="拍拍想跟你說的話")

In [33]:
iface = gr.Interface(answer_user_question,
                     title=title,
                     description=description,
                     inputs = inp,
                     outputs = [out1, out2])

In [ ]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d5d8781d2e22b36077.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
